In [1]:
import sys
import time
sys.path.append("../")
from retrieve_image_mail import EmailParser
import io
import numpy as np

def credential():
        f = open("../../data/credential.txt")
        user = f.readline().strip("\n")
        passwd = f.readline().strip("\n")
        f.close()
        return user, passwd


user_name, password = credential()

In [60]:
import datetime
datetime.datetime.today()-datetime.timedelta(days=2)

datetime.datetime(2021, 10, 26, 14, 40, 35, 299229)

In [40]:
import email
import imaplib
import datetime
from PIL import Image
import io

def get_todays_email():
    """
    Get the email metadata between to dates
    """
    if not user_name or not password:
        raise "no credential provided"
    total_nb_email = 0
    todays_email = []
    imapSession = imaplib.IMAP4_SSL('imap.gmail.com')
    typ, accountDetails = imapSession.login(user_name, password)

    if typ != 'OK':
        raise 'Not able to sign in!'
    
    total_nb_email = int(imapSession.select("INBOX")[1][0])
    typ, data = imapSession.search(None, 'ALL')
    if typ != 'OK':
        raise 'Error searching Inbox.0'
    
    typ, data = imapSession.fetch(data[0].split()[-1], '(RFC822)')
    raw_dates = data[0][1].decode('utf-8').split("\n")[7][6:-13]
    date = datetime.datetime.strptime(raw_dates, "%a, %d %b %Y %H:%M:%S")

    tz = int(data[0][1].decode('utf-8').split("\n")[7][-13:-9])
    time = datetime.datetime.strptime(data[0][1].decode('utf-8').split("\n")[7][6:-13], "%a, %d %b %Y %H:%M:%S")\
    - datetime.timedelta(hours= tz)
    print(time)

    imapSession.close()
    imapSession.logout()
    return total_nb_email, todays_email

get_todays_email()

2021-10-28 01:34:13


(986, [])

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from retrieve_image_mail import EmailParser

email_parser = EmailParser(user_name, password)
database_file = 'sqlite:////mnt/data/workspace/bird_classification/data/database.db'
engine = create_engine(database_file, echo=False)
new_data = email_parser.get_todays_email()

In [87]:
conca = np.append(np.array(last_data.iloc[:, 1]), np.array(new_data.iloc[:]))
#conca.drop(columns="index", inplace=True)
pd.DataFrame(conca).rename(columns={0: "dates"}).sort_values("dates", ascending=False).reset_index(drop=True)

,dates
0,2021-11-03 20:00:18
1,2021-11-03 16:25:50
2,2021-11-03 15:59:39
3,2021-11-03 15:59:21
4,2021-11-03 15:59:02
...,...
1271,2021-10-20 10:18:20
1272,2021-10-20 10:06:44
1273,2021-10-19 11:33:34
1274,2021-10-19 10:33:36


In [74]:
last_data

,index,dates
0,0,2021-11-01 16:23:02
1,1,2021-11-01 16:21:40
2,2,2021-11-01 16:21:01
3,3,2021-11-01 15:42:20
4,4,2021-11-01 15:40:59
...,...,...
1146,1146,2021-10-20 10:18:20
1147,1147,2021-10-20 10:06:44
1148,1148,2021-10-19 11:33:34
1149,1149,2021-10-19 10:33:36
